In [ ]:
from huggingface_hub import InferenceClient
from dotenv import load_dotenv
import os

c:\Users\ahmad\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()
key = os.getenv("HF_KEY")

In [ ]:
# Create an inference client using the Fireworks AI provider and API key
client = InferenceClient(
    provider="fireworks-ai",
    api_key=key,
)

system_prompt = """
أنت مساعد ذكي وخبير في تعليم لغة C++، وتتمثل مهمتك في إنشاء اختبارات فعّالة ومناسبة لطلاب الجامعات في مادة C++. يجب أن تلتزم بالقواعد التالية أثناء إنشاء الأسئلة:

1. يجب أن تركز الأسئلة على المفاهيم الأساسية والجوهرية ضمن المحتوى التعليمي المحدد.
2. تنوع في نوع الأسئلة: إخراج كود (output prediction)، اختيار من متعدد (multiple choice)، وتصحيح أخطاء (code correction).
3. تأكد من أن الأسئلة تغطي مفاهيم يصعب على الطلاب فهمها غالبًا، وتشمل حالات حرجة وأخطاء شائعة.
4. يجب أن تكون الأسئلة متدرجة في الصعوبة: من الأساسي إلى المتوسط، مع وضوح في صياغة السؤال.
5. استخدم أمثلة واقعية ومن سياقات برمجية مفيدة، وتجنب الأمثلة النظرية البحتة.
6. يجب أن يحتوي كل سؤال على:
   - نص السؤال بشكل واضح
   - إذا كان اختيار من متعدد: قائمة خيارات
   - المفهوم المستهدف من السؤال (concept)
7. استخدم تنسيق منسق وواضح مع مسافات بادئة مناسبة لتسهيل القراءة.
8. تجنب التكرار أو الغموض في الأسئلة.
9. استخدم اللغة العربية الفصحى، مع مصطلحات تقنية دقيقة.

هدفك هو مساعدة الطالب على تقييم فهمه الحقيقي للمادة من خلال أسئلة دقيقة، تعليمية، ومحفّزة للتفكير.
"""

examples = [
    {
        "subject": "مقدمة في البرمجة بلغة C++",
        "subject_content": """
### المفاهيم الأساسية:

1. **المتغيرات وأنواع البيانات**
   - التصريح والتخصيص
   - العمليات الحسابية (`++`, `--`, `+=`)
   - نطاق المتغير

2. **البنى الشرطية**
   - if-else المتداخلة
   - switch-case مع استخدام break

3. **الحلقات التكرارية**
   - for مع مؤشرات متعددة
   - الفرق بين while و do-while
   - التحكم في التكرار باستخدام break و continue

4. **المصفوفات**
   - المصفوفات الأحادية والثنائية
   - الوصول إلى العناصر
   - التهيئة المتداخلة
""",
        "questions": [
            {
                "type": "output_prediction",
                "difficulty": "متوسط",
                "question": "ما هو ناتج تنفيذ الكود التالي؟\n\n```cpp\nint x = 5;\ncout << (x++ * 2) << ' ' << ++x;\n```",
                "concept": "تحديثات ما بعد وما قبل الإشارة"
            },
            {
                "type": "multiple_choice",
                "difficulty": "أساسي",
                "question": "أي من التعريفات التالية صحيح لإنشاء مصفوفة ثنائية الأبعاد بحجم 2x3 في لغة C++؟",
                "options": [
                    "int arr[2][3] = {{1,2}, {3,4}, {5,6}};",
                    "int arr[2][3] = {1,2,3,4,5,6};",
                    "int arr[][] = {{1,2,3}, {4,5,6}};",
                    "int arr[3][2] = {{1,2}, {3,4}};"
                ],
                "correct": 1,
                "concept": "تهيئة المصفوفات الثنائية"
            },
            {
                "type": "code_correction",
                "difficulty": "متوسط",
                "question": "ما الخطأ في الكود التالي؟\n\n```cpp\nfor(int i = 0; i <= 5; i++); {\n    cout << i;\n}\n```",
                "concept": "الفاصلة المنقوطة الزائدة في الحلقة التكرارية"
            }
        ]
    }
]
completion = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-V3",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"""
أنت مساعد خبير في تقييم مستوى الطلاب الجامعيين بلغة C++.

بناءً على المحتوى التعليمي التالي:
{examples[0]['subject_content']}

أنشئ **3 أسئلة تقييمية متنوعة** تغطي:
- سؤال إخراج كود (Output Prediction)
- سؤال اختيار من متعدد (Multiple Choice)
- سؤال تصحيح كود (Code Correction)

**ملاحظات مهمة عند إنشاء الأسئلة:**
1. ركّز على المفاهيم الصعبة والمربكة للطلاب.
2. حاول تضمين حالات خاصة (Edge Cases) إن أمكن.
3. استخدم تنسيق كود واضح ومنظم مع مسافات بادئة مناسبة (Indented Code).
4. حدد المفهوم المستهدف (concept) لكل سؤال بوضوح.
5. استخدم أمثلة واقعية مأخوذة من تطبيقات الحياة البرمجية.

**صيغة كل سؤال يجب أن تتضمن:**
- عنوان مختصر (مثلاً: "السؤال 1: إخراج كود")
- نص السؤال بشكل واضح
- كتلة كود (إن وجد)
- خيارات (إن كان اختيار من متعدد)
- المفهوم المستهدف
        """}
    ],
    max_tokens=1000, # Limits the length of the response
    temperature=0.3,  # Controls creativity
    top_p=0.9,  # Nucleus sampling for output diversity
)

print(completion.choices[0].message.content)


### السؤال 1: إخراج كود

**نص السؤال:**  
ما هو إخراج الكود التالي؟  

```cpp
#include <iostream>
using namespace std;

int main() {
    int x = 5;
    int y = 10;
    int z = ++x + y--;
    
    cout << "x = " << x << ", y = " << y << ", z = " << z << endl;
    return 0;
}
```

**المفهوم المستهدف:**  
العمليات الحسابية (`++`, `--`) وتأثيرها على المتغيرات.

---

### السؤال 2: اختيار من متعدد

**نص السؤال:**  
ما هو الإخراج الصحيح للكود التالي؟  

```cpp
#include <iostream>
using namespace std;

int main() {
    int arr[3][3] = {{1, 2, 3}, {4, 5, 6}, {7, 8, 9}};
    cout << arr[1][2] + arr[2][0] << endl;
    return 0;
}
```

**الخيارات:**  
أ) 13  
ب) 15  
ج) 12  
د) 14  

**المفهوم المستهدف:**  
الوصول إلى عناصر المصفوفات الثنائية.

---

### السؤال 3: تصحيح كود

**نص السؤال:**  
يحتوي الكود التالي على خطأ. قم بتصحيحه بحيث يعمل بشكل صحيح ويطبع الأعداد الفردية من 1 إلى 10.  

```cpp
#include <iostream>
using namespace std;

int main() {
    for (int i = 1; i <= 10; i++) {
        if (i %